# TensorRider Algorithm

In [1]:
import numpy as np
import tensorflow as tf
import tensorlayer as tl

In [2]:
sess = tf.InteractiveSession()

In [3]:
def preprocess(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    # mono = 0.2989 * r + 0.5870 * g + 0.1140 * b
    mono = 0.5 * r + 0.25 * g + 0.25 * b
    mono = mono * (1. / 255) - 0.5
    return mono

In [4]:
def prepareDataArrays(iterator):
    X = np.zeros(4800)
    y = np.zeros(1)
    recordCounter = 0;

    for string_record in iterator:
        recordCounter += 1

        example = tf.train.Example()
        example.ParseFromString(string_record)
        imageString = (example.features.feature['image'].bytes_list.value[0])
        label = (example.features.feature['label'].int64_list.value[0])

        image = np.fromstring(imageString, dtype=np.uint8)
        image = image.reshape((60, 80, 3))
        image = preprocess(image)
        image = image.reshape((4800))

        X = np.vstack((X,image))
        y = np.append(y,label)
        if recordCounter % 100 == 0:
            print(recordCounter,end = '->')

    y = y.reshape((recordCounter + 1,))
#     y = np.round(y / 7) #Downsampling
#     y = y + 6
    y = np.round(y / 12)
    y = y + 3
    return X, y

In [5]:
trainIterator = tf.python_io.tf_record_iterator(path="train.tfrecords")
valIterator = tf.python_io.tf_record_iterator(path="val.tfrecords")
testIterator = tf.python_io.tf_record_iterator(path="test.tfrecords")

# TFRecords -> Arrays

In [6]:
print("\nTrain...")
X_train, y_train = prepareDataArrays(trainIterator)
print("\nVal...")
X_val, y_val = prepareDataArrays(valIterator)
print("\nTest...")
X_test, y_test = prepareDataArrays(testIterator)


Train...


D:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  


100->200->300->400->500->600->700->800->900->1000->1100->1200->1300->1400->1500->1600->1700->1800->1900->2000->2100->2200->2300->2400->2500->2600->2700->2800->2900->3000->3100->3200->3300->3400->3500->3600->3700->3800->3900->4000->4100->4200->4300->4400->4500->4600->4700->4800->4900->5000->5100->5200->5300->5400->5500->5600->5700->5800->5900->6000->6100->6200->6300->6400->6500->6600->6700->6800->6900->7000->7100->7200->7300->7400->7500->7600->7700->7800->7900->8000->8100->8200->8300->8400->8500->8600->8700->8800->8900->9000->9100->9200->9300->9400->9500->9600->9700->9800->9900->10000->10100->10200->10300->10400->10500->10600->10700->10800->10900->11000->11100->11200->11300->11400->11500->11600->11700->11800->11900->12000->12100->12200->12300->12400->12500->12600->12700->12800->12900->13000->13100->13200->13300->13400->13500->
Val...
100->200->300->400->500->600->700->800->900->1000->1100->1200->1300->1400->1500->1600->1700->1800->1900->2000->2100->2200->2300->2400->2500->2600->2700->
T

# Arrays -> NPZ File

In [7]:
np.savez("arrays_norm.npz", xtr = X_train, ytr = y_train, xval = X_val, yval = y_val, xt = X_test, yt = y_test)

# NPZ File -> Arrays

In [6]:
npRecall = np.load("arrays_norm.npz")
X_train = npRecall["xtr"]
y_train = npRecall["ytr"]
X_val = npRecall["xval"]
y_val = npRecall["yval"]
X_test = npRecall["xt"]
y_test = npRecall["yt"]

# Define the Neural Network

In [8]:
x = tf.placeholder(tf.float32, shape=[None, 4800], name='x')
y_ = tf.placeholder(tf.int64, shape=[None], name='y_')

In [9]:
network = tl.layers.InputLayer(x, name='input')
network = tl.layers.DropoutLayer(network, keep=0.8, name='drop1')
network = tl.layers.DenseLayer(network, 2048, tf.nn.relu, name='relu1')
# network = tl.layers.DropoutLayer(network, keep=0.5, name='drop2')
network = tl.layers.DenseLayer(network, 1024, tf.nn.relu, name='relu2')
# network = tl.layers.DropoutLayer(network, keep=0.5, name='drop3')
network = tl.layers.DenseLayer(network, 1024, tf.nn.relu, name='relu3')
network = tl.layers.DenseLayer(network, 512, tf.nn.relu, name='relu4')
# network = tl.layers.DenseLayer(network, n_units=13, act=tf.identity, name='output')
network = tl.layers.DenseLayer(network, n_units=7, act=tf.identity, name='output')

[TL] InputLayer  input: (?, 4800)
[TL] DropoutLayer drop1: keep:0.800000 is_fix:False
[TL] DenseLayer  relu1: 2048 relu
[TL] DenseLayer  relu2: 1024 relu
[TL] DenseLayer  relu3: 1024 relu
[TL] DenseLayer  relu4: 512 relu
[TL] DenseLayer  output: 7 identity


In [10]:
y = network.outputs
cost = tl.cost.cross_entropy(y, y_, name='cost')
correct_prediction = tf.equal(tf.argmax(y, 1), y_)
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
y_op = tf.argmax(tf.nn.softmax(y), 1)

In [11]:
train_params = network.all_params
train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost, var_list=train_params)

In [12]:
tl.layers.initialize_global_variables(sess)

In [13]:
network.print_params()

[TL]   param   0: relu1/W:0            (4800, 2048)       float32_ref (mean: -2.2792133677285165e-05, median: -8.772606634011026e-06, std: 0.08796939998865128)   
[TL]   param   1: relu1/b:0            (2048,)            float32_ref (mean: 0.0               , median: 0.0               , std: 0.0               )   
[TL]   param   2: relu2/W:0            (2048, 1024)       float32_ref (mean: 1.5333214378188131e-06, median: 7.346374331973493e-05, std: 0.08797972649335861)   
[TL]   param   3: relu2/b:0            (1024,)            float32_ref (mean: 0.0               , median: 0.0               , std: 0.0               )   
[TL]   param   4: relu3/W:0            (1024, 1024)       float32_ref (mean: -0.0001541113742860034, median: -0.00022260851983446628, std: 0.0879964679479599)   
[TL]   param   5: relu3/b:0            (1024,)            float32_ref (mean: 0.0               , median: 0.0               , std: 0.0               )   
[TL]   param   6: relu4/W:0            (1024, 512)     

In [14]:
network.print_layers()

[TL]   layer   0: drop1/mul:0          (?, 4800)          float32
[TL]   layer   1: relu1/Relu:0         (?, 2048)          float32
[TL]   layer   2: relu2/Relu:0         (?, 1024)          float32
[TL]   layer   3: relu3/Relu:0         (?, 1024)          float32
[TL]   layer   4: relu4/Relu:0         (?, 512)           float32
[TL]   layer   5: output/Identity:0    (?, 7)             float32


# (Optional) Setting up TensorBoard

In [13]:
acc_summ = tf.summary.scalar('acc', acc)  
cost_summ = tf.summary.scalar('cost', cost)  
summary = tf.summary.merge_all()  
writer = tf.summary.FileWriter('./logs')  
writer.add_graph(sess.graph) 

# Start Training

In [15]:
tl.utils.fit(
    sess, network, train_op, cost, X_train, y_train, x, y_, acc=acc, batch_size=256, n_epoch=500, print_freq=5, X_val=X_val, y_val=y_val, eval_train=True, tensorboard=False)

[TL] Start training the network ...
[TL] Epoch 1 of 500 took 3.618495s
[TL]    train loss: 1.274552
[TL]    train acc: 0.761129
[TL]    val loss: 2.314190
[TL]    val acc: 0.672266
[TL] Epoch 5 of 500 took 1.615907s
[TL]    train loss: 0.521170
[TL]    train acc: 0.839549
[TL]    val loss: 1.571509
[TL]    val acc: 0.669922
[TL] Epoch 10 of 500 took 1.612475s
[TL]    train loss: 0.358335
[TL]    train acc: 0.866303
[TL]    val loss: 1.312496
[TL]    val acc: 0.707422
[TL] Epoch 15 of 500 took 1.667779s
[TL]    train loss: 0.262818
[TL]    train acc: 0.897332
[TL]    val loss: 1.180094
[TL]    val acc: 0.704688
[TL] Epoch 20 of 500 took 1.714996s
[TL]    train loss: 0.231592
[TL]    train acc: 0.909714
[TL]    val loss: 1.184025
[TL]    val acc: 0.706250
[TL] Epoch 25 of 500 took 1.656025s
[TL]    train loss: 0.204195
[TL]    train acc: 0.921506
[TL]    val loss: 1.145180
[TL]    val acc: 0.699219
[TL] Epoch 30 of 500 took 1.629539s
[TL]    train loss: 0.198270
[TL]    train acc: 0.9215

KeyboardInterrupt: 

# Test the Neural Network

In [16]:
tl.utils.test(sess, network, acc, X_test, y_test, x, y_, batch_size=None, cost=cost)

[TL] Start testing the network ...
[TL]    test loss: 1.312969
[TL]    test acc: 0.700371


# Save the Trained Model

In [13]:
tl.files.save_npz(network.all_params, name='model_old.npz')
sess.close()

[TL] [*] model.npz saved


# Load the Trained Model

In [14]:
tl.files.load_and_assign_npz(sess=sess, name='model_old.npz', network=network)

[TL] [*] Load model_old.npz SUCCESS!


# Controlling TensorRider Using the Generated Model

In [15]:
import cv2
import urllib.request
import numpy as np
import socket
import threading
from time import ctime,sleep
import string

remoteImage = np.array([])
stream = urllib.request.urlopen('http://192.168.73.73:8080/?action=stream&ignored.mjpg')
bytes = bytes()

In [16]:
def Image_Refreshing_Thread():
    global remoteImage
    global stream
    global bytes
    while True:
        bytes += stream.read(1024)
        a = bytes.find(b'\xff\xd8')
        b = bytes.find(b'\xff\xd9')
        if a != -1 and b != -1:
            jpg = bytes[a:b+2]
            bytes = bytes[b+2:]
            i = cv2.imdecode(np.fromstring(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
            i = rgb2mono(i)
            remoteImage = i.reshape((1,4800))

In [17]:
def Controlling_Thread():
    global remoteImage
    addr=('192.168.73.73',51423)
    s=socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
    while True:
        direction = tl.utils.predict(sess, network, remoteImage, x, y_op, batch_size=None)
        msgCtrl_Udp = str(50) + "," + str((direction[0] - 3) * 26)
#         print(msgCtrl_Udp)
        s.sendto(msgCtrl_Udp.encode('utf-8'), addr)
        sleep(0.1)

In [18]:
RefreshImageThread = threading.Thread(target = Image_Refreshing_Thread)
RefreshImageThread.start()

D:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':


In [19]:
ControllingThread = threading.Thread(target = Controlling_Thread)
ControllingThread.start()

D:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if sys.path[0] == '':
